In [1]:
import os
import collections
import json
import pickle
from tqdm import tqdm
import ast

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.options.display.max_rows = 10

import matplotlib.pyplot as plt
import seaborn as sns

from src.wimbd_ import WimbdTasks, filter_by_language

# wt = WimbdTasks()

[nltk_data] Downloading package punkt to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


In [2]:
df_all_pth = "./results/n-grams/wmt/pile/exp4/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaTrue/2/examples_dfs_0-shot_all_models.pkl"
df_all = pd.read_pickle(df_all_pth)

df_common_pth = "./results/n-grams/wmt/pile/exp4/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaTrue/2/examples_dfs_0-shot_common_models.pkl"
df_common = pd.read_pickle(df_common_pth)

df_all_single = df_all['pythia-12b']
df_common_single = df_common['pythia-12b']

df_all_single.head(2)

,index,value,example,task,example_clean,sequence,docs,coverage,query,sum,count,id,gold,src,ref,result,bleu
0,ontario london,6770,"{'translation': {'de': 'David Bailey, Professo...",de-en,"David Bailey, Professor der University of West...",ontario london,"[train/05.jsonl.gz-4879978, train/13.jsonl.gz-...",0.591715,"{'translation': {'de': 'David Bailey, Professo...",1080974,46,2366_0,0,"Aby David Bailey, profesor Univerzity Western ...",Confirming the risks associated with the assum...,"To David Bailey, professor University Western...",22.900639
1,glass grapefruit,30,"{'translation': {'de': 'David Bailey, Professo...",de-en,"David Bailey, Professor der University of West...",glass grapefruit,"[train/22.jsonl.gz-4438103, train/12.jsonl.gz-...",0.591715,"{'translation': {'de': 'David Bailey, Professo...",1080974,46,2366_0,0,"Aby David Bailey, profesor Univerzity Western ...",Confirming the risks associated with the assum...,"To David Bailey, professor University Western...",22.900639


In [3]:
all_signgle_filter_path = df_all_pth.replace('.pkl', f'_no_english.csv')
if not os.path.exists(all_signgle_filter_path):
    df_all_single_ = filter_by_language(df_all_single, lang=None, column='index')
    df_all_single_.to_csv(all_signgle_filter_path, index=False)
else:
    df_all_single_ = pd.read_csv(all_signgle_filter_path)

In [4]:
df_all_single_['docs']

0        ['train/27.jsonl.gz-2194563', 'train/06.jsonl....
1        ['train/06.jsonl.gz-4013504', 'train/16.jsonl....
2        ['train/11.jsonl.gz-2948634', 'train/16.jsonl....
3        ['train/08.jsonl.gz-2808748', 'train/25.jsonl....
4        ['train/15.jsonl.gz-127372', 'train/11.jsonl.g...
                               ...                        
54089    ['train/28.jsonl.gz-2897119', 'train/14.jsonl....
54090    ['train/12.jsonl.gz-2318472', 'train/20.jsonl....
54091    ['train/12.jsonl.gz-5026797', 'train/02.jsonl....
54092    ['train/10.jsonl.gz-3282115', 'train/14.jsonl....
54093    ['train/28.jsonl.gz-4702027', 'train/10.jsonl....
Name: docs, Length: 54094, dtype: object

In [5]:
display(df_common['pythia-12b'])


,lang_1,lang_2,value,example,task,pair,example_clean,docs,coverage,query,sum,count,id,gold,src,ref,result,bleu
0,peut être,employee might,5,{'translation': {'en': 'We can read about seve...,fr-en,"(peut être, employee might)",Pendant qu'on peut prendre connaissance de nom...,"[train/18.jsonl.gz-66446, train/21.jsonl.gz-63...",0.072131,{'translation': {'en': 'We can read about seve...,712,13,2705_0,0,Přitom se můžeme dočíst o spoustě exemplárních...,We can read about several stories like this: t...,"On the other hand, we can read about a number...",4.445872
1,a passé,previous night,54,{'translation': {'en': 'We can read about seve...,fr-en,"(a passé, previous night)",Pendant qu'on peut prendre connaissance de nom...,"[train/07.jsonl.gz-66543, train/04.jsonl.gz-41...",0.072131,{'translation': {'en': 'We can read about seve...,712,13,2705_0,0,Přitom se můžeme dočíst o spoustě exemplárních...,We can read about several stories like this: t...,"On the other hand, we can read about a number...",4.445872
2,top management,management blog,22,{'translation': {'en': 'We can read about seve...,fr-en,"(top management, management blog)",Pendant qu'on peut prendre connaissance de nom...,"[train/09.jsonl.gz-776989, train/08.jsonl.gz-7...",0.072131,{'translation': {'en': 'We can read about seve...,712,13,2705_0,0,Přitom se můžeme dočíst o spoustě exemplárních...,We can read about several stories like this: t...,"On the other hand, we can read about a number...",4.445872
3,comme celui,another one,169,{'translation': {'en': 'We can read about seve...,fr-en,"(comme celui, another one)",Pendant qu'on peut prendre connaissance de nom...,"[train/07.jsonl.gz-5755414, train/04.jsonl.gz-...",0.072131,{'translation': {'en': 'We can read about seve...,712,13,2705_0,0,Přitom se můžeme dočíst o spoustě exemplárních...,We can read about several stories like this: t...,"On the other hand, we can read about a number...",4.445872
4,petite amie,like girl,4,{'translation': {'en': 'We can read about seve...,fr-en,"(petite amie, like girl)",Pendant qu'on peut prendre connaissance de nom...,"[train/13.jsonl.gz-1805355, train/12.jsonl.gz-...",0.072131,{'translation': {'en': 'We can read about seve...,712,13,2705_0,0,Přitom se můžeme dočíst o spoustě exemplárních...,We can read about several stories like this: t...,"On the other hand, we can read about a number...",4.445872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9091,projekt wurde,project also,1,{'translation': {'de': 'Das Projekt wurde zu e...,de-en,"(projekt wurde, project also)",Das Projekt wurde zu einer Anthologie des Flam...,[train/11.jsonl.gz-6263220],0.038317,{'translation': {'de': 'Das Projekt wurde zu e...,1,1,1252_0,0,Projekt se stal jakousi antologií flamenca a b...,The project has also become an anthology of fl...,The project has become an anthology of French...,28.436025
9092,beim verteidigungsministerium,cabinet office,1,"{'translation': {'de': 'Er sagte: ""Sie hätten ...",de-en,"(beim verteidigungsministerium, cabinet office)","Er sagte: ""Sie hätten gedacht, dass sich damit...",[train/06.jsonl.gz-2950336],0.038317,"{'translation': {'de': 'Er sagte: ""Sie hätten ...",1,1,406_0,0,"""Pravděpodobně si mysleli, že se věc vyřizuje ...","""They will have thought this was being dealt w...","""They probably thought that the matter would ...",4.899980
9093,bank wurden,bank bank,1,{'translation': {'de': 'Unter den Kunden ist b...,de-en,"(bank wurden, bank bank)",Unter den Kunden ist bislang keine Panik ausge...,[train/18.jsonl.gz-4745052],0.038317,{'translation': {'de': 'Unter den Kunden ist b...,1,1,286_0,0,Mezi klienty banky dosud nevypukla panika a po...,No panic has emerged so far among the customer...,The banks have not yet panicked and the branc...,2.717890
9094,rescate financiero,rescue package,1,{'translation': {'en': 'After several days of ...,es-en,"(rescate financiero, rescue package)",Los representantes del Congreso y de la admini...,[train/17.jsonl.gz-6168663],0.

In [6]:
def get_mass(df, doc_set):
    # Ensure doc_set is a set for efficient lookups
    doc_set = set(doc_set)
    
    # Define a function to parse 'docs' and check for intersection with doc_set
    def check_docs_intersection(docs):
        if isinstance(docs, str):
            try:
                # Safely evaluate the string to a list
                docs = ast.literal_eval(docs)
            except (ValueError, SyntaxError):
                # Handle the case where the string cannot be evaluated to a list
                print(f"Error parsing string: {docs}")
                return False
        # Check if there is any intersection between the docs and doc_set
        return any(d in doc_set for d in docs)
    
    # Calculate the mass without adding a temporary column
    mass = sum(df['docs'].apply(check_docs_intersection))
    
    return mass


def get_unique_docs(df):
    unique_docs = set()
    for docs_str in df['docs']:
        # Check if the entry is a string that needs to be converted to a list
        if isinstance(docs_str, str):
            try:
                docs_list = ast.literal_eval(docs_str)
                if isinstance(docs_list, list):  # Ensure that the result is a list
                    unique_docs.update(docs_list)
            except (ValueError, SyntaxError):
                print(f"Error parsing string: {docs_str}")
        elif isinstance(docs_str, list):
            unique_docs.update(docs_str)
        else:
            print(f"Unexpected format: {docs_str}")
    return unique_docs


def get_common_docs(df_1, df_2):
    df_1_docs_set = get_unique_docs(df_1)
    df_2_docs_set = get_unique_docs(df_2)

    common_docs = df_1_docs_set & df_2_docs_set
    uncommon_docs = df_1_docs_set ^ df_2_docs_set
    total_unique_docs = len(df_1_docs_set | df_2_docs_set)
    
    # Calculate the "mass" prevalence of the documents
    df_1_common_mass = get_mass(df_1, common_docs)
    df_2_common_mass = get_mass(df_2, common_docs)
    df_1_uncommon_mass = get_mass(df_1, uncommon_docs)
    df_2_uncommon_mass = get_mass(df_2, uncommon_docs)

    # calculate the ratio of unique documents in each set    
    df_1_set_ratio = len(df_1_docs_set) / total_unique_docs if total_unique_docs > 0 else 0
    df_2_set_ratio = len(df_2_docs_set) / total_unique_docs if total_unique_docs > 0 else 0
    
    # Calculate coverage for each set across both DataFrames
    df_1_total_coverage = (get_mass(df_1, df_1_docs_set) + get_mass(df_2, df_1_docs_set)) / (len(df_1) + len(df_2))
    df_2_total_coverage = (get_mass(df_1, df_2_docs_set) + get_mass(df_2, df_2_docs_set)) / (len(df_1) + len(df_2))
    
    # Calculate the ratio of common documents to the total unique documents
    total_unique_docs = len(df_1_docs_set | df_2_docs_set)
    common_all_ratio = len(common_docs) / total_unique_docs if total_unique_docs > 0 else 0
    
    # Print the calculated statistics
    print(f"df_1_set / total: {df_1_set_ratio}")
    print(f"df_2_set / total: {df_2_set_ratio}")
    print(f"df_1 common mass: {df_1_common_mass}")
    print(f"df_2 common mass: {df_2_common_mass}")
    print(f"df_1 uncommon mass: {df_1_uncommon_mass}")
    print(f"df_2 uncommon mass: {df_2_uncommon_mass}")
    print(f"df_1 total coverage: {df_1_total_coverage}")
    print(f"df_2 total coverage: {df_2_total_coverage}")
    print(f"common_all_ratio: {common_all_ratio}")
    
    return {
        'df_1_docs_set': df_1_docs_set,
        'df_2_docs_set': df_2_docs_set,
        'common_docs': common_docs,
        'uncommon_docs': uncommon_docs,
        'df_1_common_mass': df_1_common_mass,
        'df_2_common_mass': df_2_common_mass,
        'df_1_uncommon_mass': df_1_uncommon_mass,
        'df_2_uncommon_mass': df_2_uncommon_mass,
        'df_1_total_coverage': df_1_total_coverage,
        'df_2_total_coverage': df_2_total_coverage,
        'common_all_ratio': common_all_ratio
    }

common_docs = get_common_docs(df_all_single_, df_common_single)

df_1_set / total: 0.915605843809564
df_2_set / total: 0.12771585611176522
df_1 common mass: 22570
df_2 common mass: 7104
df_1 uncommon mass: 52695
df_2 uncommon mass: 6291
df_1 total coverage: 0.9684760246874505
df_2 total coverage: 0.501123595505618
common_all_ratio: 0.04332169992132927


In [7]:
n_display = 100
pd.set_option('display.max_columns', n_display)
pd.options.display.max_rows = n_display

display(df_all_single_[df_all_single_['task'] == 'cs-en'])

# find how many value is 0
len(df_all_single_[df_all_single_['value'] == 0])

,Unnamed: 0,index,value,example,task,example_clean,sequence,docs,coverage,query,sum,count,id,gold,src,ref,result,bleu
41,232,některých léků,4,"{'translation': {'cs': 'Aby David Bailey, prof...",cs-en,"Aby David Bailey, profesor Univerzity Western ...",některých léků,"['train/10.jsonl.gz-5235759', 'train/05.jsonl....",0.552914,"{'translation': {'cs': 'Aby David Bailey, prof...",736347,42,2366_0,0,"Aby David Bailey, profesor Univerzity Western ...",Confirming the risks associated with the assum...,"To David Bailey, professor University Western...",22.900639
42,239,stačí jedna,16,"{'translation': {'cs': 'Aby David Bailey, prof...",cs-en,"Aby David Bailey, profesor Univerzity Western ...",stačí jedna,"['train/05.jsonl.gz-2813065', 'train/12.jsonl....",0.552914,"{'translation': {'cs': 'Aby David Bailey, prof...",736347,42,2366_0,0,"Aby David Bailey, profesor Univerzity Western ...",Confirming the risks associated with the assum...,"To David Bailey, professor University Western...",22.900639
43,243,zdraví pacientů,18,"{'translation': {'cs': 'Aby David Bailey, prof...",cs-en,"Aby David Bailey, profesor Univerzity Western ...",zdraví pacientů,"['train/13.jsonl.gz-4792072', 'train/11.jsonl....",0.552914,"{'translation': {'cs': 'Aby David Bailey, prof...",736347,42,2366_0,0,"Aby David Bailey, profesor Univerzity Western ...",Confirming the risks associated with the assum...,"To David Bailey, professor University Western...",22.900639
44,246,jablečného džusu,2,"{'translation': {'cs': 'Aby David Bailey, prof...",cs-en,"Aby David Bailey, profesor Univerzity Western ...",jablečného džusu,"['train/15.jsonl.gz-3734662', 'train/08.jsonl....",0.552914,"{'translation': {'cs': 'Aby David Bailey, prof...",736347,42,2366_0,0,"Aby David Bailey, profesor Univerzity Western ...",Confirming the risks associated with the assum...,"To David Bailey, professor University Western...",22.900639
45,258,život zachraňujících,1,"{'translation': {'cs': 'Aby David Bailey, prof...",cs-en,"Aby David Bailey, profesor Univerzity Western ...",život zachraňujících,['train/01.jsonl.gz-616603'],0.552914,"{'translation': {'cs': 'Aby David Bailey, prof...",736347,42,2366_0,0,"Aby David Bailey, profesor Univerzity Western ...",Confirming the risks associated with the assum...,"To David Bailey, professor University Western...",22.900639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54079,196832,jeho nevlastní,10,{'translation': {'cs': 'Jeho nevlastní dcera š...,cs-en,Jeho nevlastní dcera šla do Plymouthu. His ste...,jeho nevlastní,"['train/28.jsonl.gz-4101446', 'train/00.jsonl....",0.552914,{'translation': {'cs': 'Jeho nevlastní dcera š...,13,2,665_0,0,Jeho nevlastní dcera šla do Plymouthu.,His step-daughter went to Plymouth.,His own daughter went to Plymouth.,43.472087
54080,196833,nemyslíš zaplatíš,12,"{'translation': {'cs': 'Nemyslíš - zaplatíš!',...",cs-en,Nemyslíš - zaplatíš! You don't think - you pay!,nemyslíš zaplatíš,"['train/17.jsonl.gz-6742461', 'train/25.jsonl....",0.552914,"{'translation': {'cs': 'Nemyslíš - zaplatíš!',...",12,1,416_0,0,Nemyslíš - zaplatíš!,You don't think - you pay!,You don't think - you pay!,100.000000
54083,196862,pane byla,2,"{'translation': {'cs': 'Pane, já byla nervózní...",cs-en,"Pane, já byla nervózní. I was so excited.",pane byla,"['train/01.jsonl.gz-1921155', 'train/02.jsonl....",0.552914,"{'translation': {'cs': 'Pane, já byla nervózní...",4,2,2864_0,0,"Pane, já byla nervózní.",I was so excited.,I was nervous.,27.534766
54084,196863,byla nervózní,2,"{'translation': {'cs': 'Pane, já byla nervózní...",cs-en,"Pane, já byla nervózní. I was so excited.",byla nervózní,"['train/05.jsonl.gz-5817784', 'train/08.jsonl....",0.552914,"{'translation': {'cs': 'Pane, já byla nervózní...",4,2,2864_0,0,"Pane, já byla nervózní.",I was so excited.,I was nervous.,27.534766


0

In [8]:
df_all_single_['docs']

0        ['train/27.jsonl.gz-2194563', 'train/06.jsonl....
1        ['train/06.jsonl.gz-4013504', 'train/16.jsonl....
2        ['train/11.jsonl.gz-2948634', 'train/16.jsonl....
3        ['train/08.jsonl.gz-2808748', 'train/25.jsonl....
4        ['train/15.jsonl.gz-127372', 'train/11.jsonl.g...
                               ...                        
54089    ['train/28.jsonl.gz-2897119', 'train/14.jsonl....
54090    ['train/12.jsonl.gz-2318472', 'train/20.jsonl....
54091    ['train/12.jsonl.gz-5026797', 'train/02.jsonl....
54092    ['train/10.jsonl.gz-3282115', 'train/14.jsonl....
54093    ['train/28.jsonl.gz-4702027', 'train/10.jsonl....
Name: docs, Length: 54094, dtype: object

In [9]:
len(df_all_single_), len(df_common_single)

(54094, 9096)

In [10]:
doc_set = common_docs['df_1_docs_set']
mass = get_mass(df_all_single_, doc_set)

In [11]:
mass

54094